<a href="https://colab.research.google.com/github/TheClub4/IMDB_Sentiment_Analysis/blob/master/IMDB_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='greyblue'>Logistic Regression: A Sentimental Analysis Case Study</font>

## Task 1: Loading the dataset
_______

In [26]:
import pandas as pd
df = pd.read_csv('movie_data.csv')


df.head(10)

,text,sentiment
0,For a movie that gets no respect there sure ar...,0
1,Bizarre horror movie filled with famous faces ...,0
2,"A solid, if unremarkable film. Matthau, as Ein...",0
3,It's a strange feeling to sit alone in a theat...,0
4,"You probably all already know this by now, but...",0
5,I saw the movie with two grown children. Altho...,0
6,You're using the IMDb. You've given some heft...,0
7,This was a good film with a powerful message o...,0
8,"Made after QUARTET was, TRIO continued the qua...",0
9,"For a mature man, to admit that he shed a tear...",0


In [27]:
df['text'][0]

'For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.'

# Task 2: Data Cleaning
___

In [0]:
import re
def preprocessor(text):
  text =re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
  
  return text

In [20]:
preprocessor("This is a :) test :-( !")

'this is a test :) :('

In [0]:
# preprocess train and test data respectively 
df['text'] = df['text'].apply(preprocessor)


# Task 3: Tokenization
___

In [0]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

In [0]:
def tokenizer(text):
  return text.split()

In [0]:
def tokenizer_stemmer(text):
  return[porter.stem(word) for word in text.split()]

In [10]:
tokenizer('runners like running thus they run')

['runners', 'like', 'running', 'thus', 'they', 'run']

In [11]:
tokenizer_stemmer('runners like running thus they run')

['runner', 'like', 'run', 'thu', 'they', 'run']

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_stemmer('runners like running thus they run') if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

# Task 4: Transform Text Data into TF-IDF Vectors
___

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                         lowercase=True,
                         preprocessor=None,  # applied preprocessor in Data Cleaning
                         tokenizer=tokenizer_stemmer,
                         use_idf=True,
                         norm='l2',
                         smooth_idf=True)

y = df.sentiment.values
X = tfidf.fit_transform(df.text)

# Task 5: Document Classification using Logistic Regression
___

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, shuffle=False)

In [31]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=0,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.5min finished


In [0]:
saved_model = open('saved_model.sav', 'wb')
pickle.dump(clf, saved_model)
saved_model.close()

# Task 6: Model Evaluation
___

In [0]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [36]:
saved_clf.score(X_test, y_test)

0.9448